<div style="background: linear-gradient(90deg, #1e3c72 0%, #2a5298 100%); color: white; padding: 32px 24px; border-radius: 12px; box-shadow: 0 2px 8px rgba(30,60,114,0.15); margin-bottom: 24px; text-align: center;">
  <h1 style="font-size: 2.5em; font-weight: 700; margin-bottom: 0.2em; letter-spacing: 1px;">SBS Evaluation Tool - Complete Demo</h1>
  <h3 style="font-weight: 400; margin-bottom: 1em;">Полная демонстрация всех модулей M0-M5</h3>
  <p style="font-size: 1.15em; max-width: 900px; margin: 0 auto;">
    Этот ноутбук демонстрирует <b>все модули тестирования</b> от <b>M0 до M5</b> на реальных данных с использованием <b>CatBoost</b>.<br>
    Покрывает <b>качество данных, распределения, Gini, мультиколлинеарность, производительность модели и стабильность</b>.<br>
    Каждый тест создает детальный HTML отчет с светофорными сигналами и рекомендациями для production.
  </p>
  <hr style="border: none; border-top: 1px solid #fff; margin: 24px 0;">
  <div style="display: flex; flex-wrap: wrap; justify-content: center; gap: 32px;">
    <div style="min-width: 240px;">
      <h4 style="margin-bottom: 0.5em;">Содержание</h4>
      <ul style="text-align: left; font-size: 1.05em; line-height: 1.6;">
        <li>1. Импорт библиотек</li>
        <li>2. Загрузка и исследование данных</li>
        <li>3. Предобработка данных</li>
        <li>4. Обучение CatBoost моделей</li>
        <li>5. M0: Качество данных</li>
        <li>6. M1: Тесты распределения</li>
        <li>7. M2: Gini тесты</li>
        <li>8. M3: Мультиколлинеарность</li>
        <li>9. M4: Производительность модели</li>
        <li>10. M5: Стабильность модели</li>
        <li>11. Объединенный отчет</li>
        <li>12. Заключение</li>
      </ul>
    </div>
    <div style="min-width: 240px;">
      <h4 style="margin-bottom: 0.5em;">Модули тестирования</h4>
      <ul style="text-align: left; font-size: 1.05em; line-height: 1.6;">
        <li><b>M0:</b> Качество данных (5 тестов)</li>
        <li><b>M1:</b> Распределения (3 теста)</li>
        <li><b>M2:</b> Gini анализ (3 теста)</li>
        <li><b>M3:</b> Мультиколлинеарность (5 тестов)</li>
        <li><b>M4:</b> Производительность (14 тестов)</li>
        <li><b>M5:</b> Стабильность (5 тестов)</li>
        <li><b>Итого:</b> 35 тестов</li>
      </ul>
    </div>
  </div>
</div>

---

# О данном ноутбуке

Этот ноутбук предназначен для <b>data scientists, ML engineers, и аналитиков рисков</b>, которым необходимо валидировать модели в production. Он предоставляет:
- Пошаговую демонстрацию всех 35 тестов из 5 модулей
- Визуальную и статистическую диагностику качества данных и моделей  
- Автоматическую генерацию отчетов для аудита и документации
- Практические советы по интеграции тестов в рабочий процесс

<b>Используемые датасеты:</b>
- <b>Titanic</b> (бинарная классификация) 
- <b>California Housing</b> (регрессия)

<b>Технологии:</b> CatBoost, scikit-learn, pandas, numpy, matplotlib, seaborn, SHAP

<b>Автор:</b> SBS Evaluation Tool Team  
<b>Дата:</b> Август 2025

## 1. Импорт необходимых библиотек

In [2]:
# Основные библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')
from catboost import CatBoostClassifier, CatBoostRegressor

# ML библиотеки
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score, roc_curve, classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.datasets import fetch_california_housing
from sklearn.calibration import calibration_curve  # <-- правильный импорт
import shap

# Наши тесты - исправленные импорты
import sys
import os

# Добавляем текущую директорию в путь
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)

# Импортируем базовые классы
from core.base_test import ModelReportBuilder, TestGroupBuilder

# Импортируем все доступные тесты M0 - качество данных
from tests.m0_data_quality import (
    M01_DataSummaryTest, M02_SampleOverlayTest, M03_ObsVsPredTest
)

# Импортируем все доступные тесты M1 - тесты распределения
from tests.m1_distribution_tests import (
    M11_TrainVsGenpopPSITest, M12_OOSVsGenpopPSITest, M13_MaxDateRecencyTest,
    M14_DataQualityOverviewTest, M15_MissingValuesTest
)

# Импортируем все доступные тесты M2 - Gini тесты  
from tests.m2_gini_tests import (
    M21_GiniTest, M22_GiniFactorsTest, M24_GiniDynamicsTest, M25_GiniUpliftTest
)

# Импортируем все доступные тесты M3 - мультиколлинеарность
from tests.m3_multicollinearity import (
    M33_MulticollinearityTest  # M31 помечен как NOT FOR USE
)

# Импортируем все тесты M4
from tests.m4_model_performance import (
    M41_MetricConfidenceIntervalTest, M42_SampleSizeAdequacyTest, M43_CalibrationTest,
    M44_OverfittingTest, M45_MetricStabilityTest, M46_CategoryQualityTest,
    M47_ShapImportanceTest, M48_FeatureContributionTest, M49_UpliftTest,
    M410_MissingValuesImpactTest, M411_TargetCorrelationTest, M412_FeatureCorrelationTest,
    M413_VIFTest, M414_TwoForestSelectionTest
)

# Импортируем все доступные тесты M5 - стабильность модели
from tests.m5_model_stability import (
    M51_ModelGiniStabilityTest, M52_FactorGiniStabilityTest, M53_ScorePSITest, M54_FactorPSITest
)

# Импортируем PSI калькулятор
from tests.psi_calculator import PSICalculator, PSICalculatorExtended

print("Все библиотеки успешно импортированы!")
print("Доступны тесты:")
print("• M0 (качество данных): M01_DataSummaryTest, M02_SampleOverlayTest, M03_ObsVsPredTest")
print("• M1 (распределения): M11_TrainVsGenpopPSITest, M12_OOSVsGenpopPSITest, M13_MaxDateRecencyTest, M14_DataQualityOverviewTest, M15_MissingValuesTest")
print("• M2 (Gini): M21_GiniTest, M22_GiniFactorsTest, M24_GiniDynamicsTest, M25_GiniUpliftTest")
print("• M3 (мультиколлинеарность): M33_MulticollinearityTest")
print("• M4 (модели): 15 тестов")
print("• M5 (стабильность): M51_ModelGiniStabilityTest, M52_FactorGiniStabilityTest, M53_ScorePSITest, M54_FactorPSITest")

Все библиотеки успешно импортированы!
Доступны тесты:
• M0 (качество данных): M01_DataSummaryTest, M02_SampleOverlayTest, M03_ObsVsPredTest
• M1 (распределения): M11_TrainVsGenpopPSITest, M12_OOSVsGenpopPSITest, M13_MaxDateRecencyTest, M14_DataQualityOverviewTest, M15_MissingValuesTest
• M2 (Gini): M21_GiniTest, M22_GiniFactorsTest, M24_GiniDynamicsTest, M25_GiniUpliftTest
• M3 (мультиколлинеарность): M33_MulticollinearityTest
• M4 (модели): 15 тестов
• M5 (стабильность): M51_ModelGiniStabilityTest, M52_FactorGiniStabilityTest, M53_ScorePSITest, M54_FactorPSITest


## 2. Загрузка и исследование датасетов

### 2.1 Датасет Titanic для бинарной классификации

In [3]:
# Загрузка датасета Titanic
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
titanic_df = pd.read_csv(url)

print(f"Размер датасета Titanic: {titanic_df.shape}")
print("\nПервые 5 строк:")
titanic_df.head()

Размер датасета Titanic: (891, 12)

Первые 5 строк:


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# Исследование данных Titanic
print("Информация о датасете:")
print(titanic_df.info())
print("\nПропущенные значения:")
print(titanic_df.isnull().sum())
print("\nРаспределение целевой переменной:")
print(titanic_df['Survived'].value_counts())

Информация о датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None

Пропущенные значения:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cab

### 2.2 Датасет California Housing для регрессии

In [5]:
# Загрузка датасета California Housing
housing_data = fetch_california_housing()
housing_df = pd.DataFrame(housing_data.data, columns=housing_data.feature_names)
housing_df['target'] = housing_data.target

print(f"Размер датасета California Housing: {housing_df.shape}")
print("\nПервые 5 строк:")
housing_df.head()

Размер датасета California Housing: (20640, 9)

Первые 5 строк:


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [6]:
# Исследование данных California Housing
print("Информация о датасете:")
print(housing_df.info())
print("\nОписательная статистика:")
housing_df.describe()

Информация о датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   MedInc      20640 non-null  float64
 1   HouseAge    20640 non-null  float64
 2   AveRooms    20640 non-null  float64
 3   AveBedrms   20640 non-null  float64
 4   Population  20640 non-null  float64
 5   AveOccup    20640 non-null  float64
 6   Latitude    20640 non-null  float64
 7   Longitude   20640 non-null  float64
 8   target      20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB
None

Описательная статистика:


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704,2.068558
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532,1.153956
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000,0.149990
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000,1.196000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000,1.797000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000,2.647250
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000,5.000010


## 3. Предобработка данных

### 3.1 Предобработка данных Titanic

In [7]:
# Предобработка Titanic
def preprocess_titanic(df):
    df = df.copy()
    
    # Заполнение пропущенных значений
    df['Age'].fillna(df['Age'].median(), inplace=True)
    df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
    df['Fare'].fillna(df['Fare'].median(), inplace=True)
    
    # Создание новых признаков
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
    df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    
    # Группировка редких титулов
    title_mapping = {
        'Mr': 0, 'Miss': 1, 'Mrs': 2, 'Master': 3, 'Dr': 4, 'Rev': 4, 'Col': 4, 
        'Major': 4, 'Mlle': 1, 'Countess': 4, 'Ms': 1, 'Lady': 4, 'Jonkheer': 4, 
        'Don': 4, 'Dona': 4, 'Mme': 2, 'Capt': 4, 'Sir': 4
    }
    df['Title'] = df['Title'].map(title_mapping).fillna(4)
    
    # Кодирование категориальных признаков
    le_sex = LabelEncoder()
    df['Sex_encoded'] = le_sex.fit_transform(df['Sex'])
    
    le_embarked = LabelEncoder()
    df['Embarked_encoded'] = le_embarked.fit_transform(df['Embarked'])
    
    # Выбор признаков для модели
    features = ['Pclass', 'Sex_encoded', 'Age', 'Fare', 'Embarked_encoded', 
                'FamilySize', 'IsAlone', 'Title']
    
    return df[features], df['Survived']

X_titanic, y_titanic = preprocess_titanic(titanic_df)
print(f"Размер обработанных данных Titanic: {X_titanic.shape}")
print("Признаки:", X_titanic.columns.tolist())

Размер обработанных данных Titanic: (891, 8)
Признаки: ['Pclass', 'Sex_encoded', 'Age', 'Fare', 'Embarked_encoded', 'FamilySize', 'IsAlone', 'Title']


### 3.2 Предобработка данных California Housing

In [8]:
# Предобработка California Housing
def preprocess_housing(df):
    df = df.copy()
    
    # Создание новых признаков
    df['rooms_per_household'] = df['AveRooms'] / df['AveOccup']
    df['bedrooms_per_room'] = df['AveBedrms'] / df['AveRooms']
    df['population_per_household'] = df['Population'] / df['HouseAge']
    
    # Логарифмирование некоторых признаков
    df['log_MedInc'] = np.log1p(df['MedInc'])
    df['log_Population'] = np.log1p(df['Population'])
    
    features = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 
                'AveOccup', 'Latitude', 'Longitude', 'rooms_per_household',
                'bedrooms_per_room', 'population_per_household', 'log_MedInc', 'log_Population']
    
    X = df[features]
    y = df['target']
    
    return X, y

X_housing, y_housing = preprocess_housing(housing_df)
print(f"Размер обработанных данных Housing: {X_housing.shape}")
print("Признаки:", X_housing.columns.tolist())

Размер обработанных данных Housing: (20640, 13)
Признаки: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude', 'rooms_per_household', 'bedrooms_per_room', 'population_per_household', 'log_MedInc', 'log_Population']


## 4. Обучение CatBoost для бинарной классификации

In [9]:
# Разделение данных Titanic
X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(
    X_titanic, y_titanic, test_size=0.2, random_state=42, stratify=y_titanic
)

print(f"Размер обучающей выборки: {X_train_cls.shape}")
print(f"Размер тестовой выборки: {X_test_cls.shape}")

Размер обучающей выборки: (712, 8)
Размер тестовой выборки: (179, 8)


In [10]:
# Обучение CatBoost классификатора
catboost_cls = CatBoostClassifier(
    iterations=500,
    learning_rate=0.1,
    depth=6,
    random_seed=42,
    verbose=False
)

catboost_cls.fit(X_train_cls, y_train_cls)

# Предсказания
y_pred_cls = catboost_cls.predict(X_test_cls)
y_pred_proba_cls = catboost_cls.predict_proba(X_test_cls)[:, 1]

print("Модель классификации обучена!")
print("\nОтчет по классификации:")
print(classification_report(y_test_cls, y_pred_cls))

Модель классификации обучена!

Отчет по классификации:
              precision    recall  f1-score   support

           0       0.87      0.88      0.88       110
           1       0.81      0.80      0.80        69

    accuracy                           0.85       179
   macro avg       0.84      0.84      0.84       179
weighted avg       0.85      0.85      0.85       179



## 5. Обучение CatBoost для регрессии

In [11]:
# Разделение данных Housing
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_housing, y_housing, test_size=0.2, random_state=42
)

print(f"Размер обучающей выборки: {X_train_reg.shape}")
print(f"Размер тестовой выборки: {X_test_reg.shape}")

Размер обучающей выборки: (16512, 13)
Размер тестовой выборки: (4128, 13)


In [12]:
# Обучение CatBoost регрессора
catboost_reg = CatBoostRegressor(
    iterations=500,
    learning_rate=0.1,
    depth=6,
    random_seed=42,
    verbose=False
)

catboost_reg.fit(X_train_reg, y_train_reg)

# Предсказания
y_pred_reg = catboost_reg.predict(X_test_reg)

print("Модель регрессии обучена!")
print(f"\nR² Score: {r2_score(y_test_reg, y_pred_reg):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test_reg, y_pred_reg)):.4f}")
print(f"MAE: {mean_absolute_error(y_test_reg, y_pred_reg):.4f}")

Модель регрессии обучена!

R² Score: 0.8452
RMSE: 0.4504
MAE: 0.2957


## 6. Демонстрация всех тестов M0-M5

### 6.1 Модуль M0 - Качество данных

In [19]:

# Объединение train и test данных регрессии в единый DataFrame с метками и датой
import pandas as pd
import numpy as np

# Получим предсказания и для обучающей выборки (полезно для анализа переобучения)
y_pred_train_reg = catboost_reg.predict(X_train_reg)

# --- Обучающая выборка (train) ---
df_train_reg = X_train_reg.copy()
df_train_reg['target'] = y_train_reg
df_train_reg['score'] = y_pred_train_reg
df_train_reg['sample'] = 'train'
# Генерируем случайные даты для train в диапазоне 6 месяцев
train_dates = pd.to_datetime(np.random.choice(pd.date_range('2023-07-01', '2024-12-31'), len(df_train_reg)))
df_train_reg['date'] = train_dates

# --- Тестовая выборка (test) ---
df_test_reg = X_test_reg.copy()
df_test_reg['target'] = y_test_reg
df_test_reg['score'] = y_pred_reg
df_test_reg['sample'] = 'test'
# Генерируем случайные даты для test в последующем диапазоне 2 месяцев
test_dates = pd.to_datetime(np.random.choice(pd.date_range('2025-01-01', '2025-05-28'), len(df_test_reg)))
df_test_reg['date'] = test_dates

# --- Объединение и создание ID ---
df_reg = pd.concat([df_train_reg, df_test_reg]).sort_values('date').reset_index(drop=True)
df_reg['id'] = df_reg.index # Добавляем первичный ключ

print("Объединенный датафрейм для регрессии готов.")
print(f"Размер: {df_reg.shape}")
print("\nПример данных (начало):")
print(df_reg.head())
print("\nПример данных (конец):")
print(df_reg.tail())
print("\nИнформация о типах и датах:")
df_reg.info()

Объединенный датафрейм для регрессии готов.
Размер: (20640, 18)

Пример данных (начало):
   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  2.9744      43.0  6.070064   1.041401       996.0  3.171975     38.26   
1  3.8676      40.0  5.514196   1.003155       914.0  2.883281     37.67   
2  3.1275      29.0  4.501564   1.081335      2102.0  2.191867     33.80   
3  2.1333      31.0  3.930657   1.094891      1198.0  4.372263     34.07   
4  1.7885      40.0  5.916817   1.198915      1467.0  2.652803     41.78   

   Longitude  rooms_per_household  bedrooms_per_room  \
0    -121.56             1.913655           0.171563   
1    -122.13             1.912473           0.181922   
2    -118.32             2.053758           0.240213   
3    -118.21             0.898998           0.278552   
4    -121.62             2.230402           0.202628   

   population_per_household  log_MedInc  log_Population  target     score  \
0                 23.162791    1.379874 

In [21]:
y_pred_cls = catboost_cls.predict(X_test_cls)
y_pred_proba_cls = catboost_cls.predict_proba(X_test_cls)[:, 1]

# Предсказания для обучающей выборки (для анализа переобучения и тестов)
y_pred_train_cls = catboost_cls.predict(X_train_cls)
y_pred_proba_train_cls = catboost_cls.predict_proba(X_train_cls)[:, 1]


In [22]:
# --- Обучающая выборка (train) ---
df_train_cls = X_train_cls.copy()
df_train_cls['target'] = y_train_cls
df_train_cls['score'] = y_pred_proba_train_cls
df_train_cls['prediction'] = y_pred_train_cls
df_train_cls['sample'] = 'train'
# Генерируем случайные даты для train
train_dates_cls = pd.to_datetime(np.random.choice(pd.date_range('2024-07-01', '2024-12-31'), len(df_train_cls)))
df_train_cls['date'] = train_dates_cls

# --- Тестовая выборка (test) ---
df_test_cls = X_test_cls.copy()
df_test_cls['target'] = y_test_cls
df_test_cls['score'] = y_pred_proba_cls
df_test_cls['prediction'] = y_pred_cls
df_test_cls['sample'] = 'test'
# Генерируем случайные даты для test
test_dates_cls = pd.to_datetime(np.random.choice(pd.date_range('2025-01-01', '2025-02-28'), len(df_test_cls)))
df_test_cls['date'] = test_dates_cls

# --- Объединение и создание ID ---
df_cls = pd.concat([df_train_cls, df_test_cls]).sort_values('date').reset_index(drop=True)
df_cls['id'] = df_cls.index

print("Объединенный датафрейм для классификации готов.")
print(f"Размер: {df_cls.shape}")
print("\nПример данных:")
print(df_cls.head())
print("\nИнформация о типах и датах:")
df_cls.info()

Объединенный датафрейм для классификации готов.
Размер: (891, 14)

Пример данных:
   Pclass  Sex_encoded   Age     Fare  Embarked_encoded  FamilySize  IsAlone  \
0       3            0  22.0   7.7500                 1           1        1   
1       2            1  51.0  12.5250                 2           1        1   
2       2            1  23.0  15.0458                 0           1        1   
3       3            0   2.0  12.2875                 2           2        0   
4       1            1  28.0  26.0000                 2           1        1   

   Title  target     score  prediction sample       date  id  
0      1       1  0.984229           1  train 2024-07-01   0  
1      4       0  0.019257           0  train 2024-07-01   1  
2      0       0  0.025245           0  train 2024-07-01   2  
3      1       1  0.971222           1  train 2024-07-02   3  
4      0       0  0.052876           0  train 2024-07-02   4  

Информация о типах и датах:
<class 'pandas.core.frame.Data

In [23]:
from tests.m0_data_quality import M01_DataSummaryTest, M02_SampleOverlayTest, M03_ObsVsPredTest
from core.base_test import TestGroupBuilder

# Пример данных (замените на ваши переменные)
df = df_cls  # основной датафрейм
date_column = 'date'  # колонка с датой
sample_column = 'sample'  # колонка с пометкой train/test
primary_key = 'id'  # первичный ключ
predict_column = 'score'  # колонка с предсказаниями
target_column = 'target'  # колонка с фактом

# 2. Создание группы
group_m0 = TestGroupBuilder(group_code="M0", group_name="M 0. Data Quality Tests")
group_m0.add_test(M01_DataSummaryTest())
group_m0.add_test(M02_SampleOverlayTest())
group_m0.add_test(M03_ObsVsPredTest())

# 3. Запуск всех тестов с передачей параметров
group_m0.run_all_tests(test_params={
    "M 0.1": {
        "df": df,
        "date_column": date_column,
        "sample_column": sample_column,
    },
    "M 0.2": {
        "df": df,
        "primary_key": primary_key,
        "sample_column": sample_column,
    },
    "M 0.3": {
        "df": df,
        "predict_column": predict_column,
        "target_column": target_column,
        "sample_column": sample_column,
    }
})

### 6.2 Модуль M1 - Тесты распределения

In [ ]:
# ===================================================================
# Группа M1 - Тесты распределения (на примере данных классификации)
# ===================================================================
from tests.psi_calculator import PSICalculatorExtended
psi_calc = PSICalculatorExtended()

print("\n--- Запуск группы M1: Тесты распределения ---")

# Определяем переменные для удобства
df_train = df_cls[df_cls['sample'] == 'train']
df_test = df_cls[df_cls['sample'] == 'test']

# Создаем группу
group_m1 = TestGroupBuilder(group_code="M1", group_name="M1. Тесты распределения")
group_m1.add_test(M11_TrainVsGenpopPSITest(psi_calc=psi_calc))
group_m1.add_test(M12_OOSVsGenpopPSITest(psi_calc=psi_calc))
group_m1.add_test(M13_MaxDateRecencyTest())
group_m1.add_test(M14_DataQualityOverviewTest())
group_m1.add_test(M15_MissingValuesTest())

# Запускаем тесты
group_m1.run_all_tests(test_params={
    "M 1.1": {
        "df_train": df_train,
        "df_test": df_test,
        "features": features_cls,
        "cat_features": cat_features
    },
    "M 1.2": {
        "df_train": df_train,
        "df_test": df_test,
        "features": features_cls
    },
    "M 1.3": {
        "df": df_train,
        "features": features_cls
    },
    "M 1.5": {
        "df": df_cls,
        "features": features_cls
    }
})

# Добавляем группу в отчет
report_builder.add_group(group_m1)
print("Группа M1 завершена.")

In [14]:
# Создаем PSI калькулятор для тестов, которые его требуют
from tests.psi_calculator import PSICalculatorExtended
psi_calc = PSICalculatorExtended()

# M1.1 - PSI тест обучения против генеральной совокупности
print("=== M1.1: PSI тест обучения против генеральной совокупности ===")

test_m11 = M11_TrainVsGenpopPSITest(psi_calc=psi_calc)
try:
    # Создаем данные с целевой переменной
    train_df = X_train_cls.copy()
    train_df['target'] = y_train_cls
    
    genpop_df = X_test_cls.copy()
    genpop_df['target'] = y_test_cls
    
    result_m11 = test_m11.run(
        train_df=train_df, 
        genpop_df=genpop_df, 
        target_column='target',
        features=['Pclass', 'Sex_encoded', 'Age', 'Fare'],
        categorical_features=['Pclass', 'Sex_encoded']
    )
    with open('m11_train_vs_genpop_psi_report.html', 'w', encoding='utf-8') as f:
        f.write(result_m11['DASHBOARD'])
    print(f"Тест M1.1 завершен. Сигнал: {test_m11.test_signal}. Отчет сохранен в m11_train_vs_genpop_psi_report.html")
except Exception as e:
    print(f"Ошибка в тесте M1.1: {e}")
    print("Пропускаем этот тест...")

# M1.2 - PSI тест out-of-sample против генеральной совокупности
print("\n=== M1.2: PSI тест out-of-sample против генеральной совокупности ===")

test_m12 = M12_OOSVsGenpopPSITest(psi_calc=psi_calc)
try:
    result_m12 = test_m12.run(
        oot_df=genpop_df,  # используем test как out-of-time
        genpop_df=train_df,  # используем train как genpop
        target_column='target',
        features=['Pclass', 'Sex_encoded', 'Age', 'Fare'],
        categorical_features=['Pclass', 'Sex_encoded']
    )
    with open('m12_oos_vs_genpop_psi_report.html', 'w', encoding='utf-8') as f:
        f.write(result_m12['DASHBOARD'])
    print(f"Тест M1.2 завершен. Сигнал: {test_m12.test_signal}. Отчет сохранен в m12_oos_vs_genpop_psi_report.html")
except Exception as e:
    print(f"Ошибка в тесте M1.2: {e}")
    print("Пропускаем этот тест...")

# M1.3 - Тест актуальности максимальной даты
print("\n=== M1.3: Тест актуальности максимальной даты ===")

test_m13 = M13_MaxDateRecencyTest()
try:
    # Создаем данные с датами для демонстрации
    data_with_dates = X_test_cls.copy()
    data_with_dates['date_column'] = pd.date_range('2023-01-01', periods=len(data_with_dates), freq='D')
    
    result_m13 = test_m13.run(df=data_with_dates, date_column='date_column')
    with open('m13_max_date_recency_report.html', 'w', encoding='utf-8') as f:
        f.write(result_m13['date_recency'])
    print(f"Тест M1.3 завершен. Сигнал: {test_m13.test_signal}. Отчет сохранен в m13_max_date_recency_report.html")
except Exception as e:
    print(f"Ошибка в тесте M1.3: {e}")
    print("Пропускаем этот тест...")

# M1.4 - Обзор качества данных
print("\n=== M1.4: Обзор качества данных ===")

test_m14 = M14_DataQualityOverviewTest()
try:
    # Создаем данные для демонстрации M1.4
    demo_data = X_test_cls.copy()
    demo_data['date_column'] = pd.date_range('2023-01-01', periods=len(demo_data), freq='D')
    demo_data['sample_column'] = ['TRAIN'] * (len(demo_data)//2) + ['TEST'] * (len(demo_data) - len(demo_data)//2)
    demo_data['primary_key'] = range(len(demo_data))
    demo_data['predict_column'] = y_pred_proba_cls[:len(demo_data)]
    demo_data['target_column'] = y_test_cls[:len(demo_data)]
    
    result_m14 = test_m14.run(
        df=demo_data,
        date_column='date_column',
        sample_column='sample_column',
        primary_key='primary_key',
        predict_column='predict_column',
        target_column='target_column',
        valid_samples=['TRAIN', 'TEST']
    )
    with open('m14_data_quality_overview_report.html', 'w', encoding='utf-8') as f:
        f.write(result_m14['overview'])
    print(f"Тест M1.4 завершен. Сигнал: {test_m14.test_signal}. Отчет сохранен в m14_data_quality_overview_report.html")
except Exception as e:
    print(f"Ошибка в тесте M1.4: {e}")
    print("Пропускаем этот тест...")

# M1.5 - Тест пропущенных значений
print("\n=== M1.5: Тест пропущенных значений ===")

test_m15 = M15_MissingValuesTest()
try:
    result_m15 = test_m15.run(
        df=X_test_cls, 
        features=['Pclass', 'Sex_encoded', 'Age', 'Fare']
    )
    with open('m15_missing_values_report.html', 'w', encoding='utf-8') as f:
        f.write(result_m15['missing_values'])
    print(f"Тест M1.5 завершен. Сигнал: {test_m15.test_signal}. Отчет сохранен в m15_missing_values_report.html")
except Exception as e:
    print(f"Ошибка в тесте M1.5: {e}")
    print("Пропускаем этот тест...")

=== M1.1: PSI тест обучения против генеральной совокупности ===


M 1.1 - Processing Features: 100%|██████████| 4/4 [00:02<00:00,  1.71it/s]


Тест M1.1 завершен. Сигнал: green. Отчет сохранен в m11_train_vs_genpop_psi_report.html

=== M1.2: PSI тест out-of-sample против генеральной совокупности ===


M 1.2 - Processing Features: 100%|██████████| 4/4 [00:02<00:00,  1.93it/s]

Тест M1.2 завершен. Сигнал: green. Отчет сохранен в m12_oos_vs_genpop_psi_report.html

=== M1.3: Тест актуальности максимальной даты ===
Тест M1.3 завершен. Сигнал: red. Отчет сохранен в m13_max_date_recency_report.html

=== M1.4: Обзор качества данных ===
Тест M1.4 завершен. Сигнал: green. Отчет сохранен в m14_data_quality_overview_report.html

=== M1.5: Тест пропущенных значений ===
Тест M1.5 завершен. Сигнал: green. Отчет сохранен в m15_missing_values_report.html


### 6.3 Модуль M2 - Gini тесты

In [15]:
# Сначала получаем предсказания для train выборки (нужно для M2 тестов)
y_pred_proba_train_cls = catboost_cls.predict_proba(X_train_cls)[:, 1]

# M2.1 - Gini тест
print("=== M2.1: Gini тест ===")

test_m21 = M21_GiniTest()
try:
    # M21_GiniTest.run() требует score_train, target_train, score_test, target_test
    result_m21 = test_m21.run(
        score_train=pd.Series(y_pred_proba_train_cls), 
        target_train=y_train_cls,
        score_test=pd.Series(y_pred_proba_cls),
        target_test=y_test_cls
    )
    with open('m21_gini_test_report.html', 'w', encoding='utf-8') as f:
        f.write(result_m21['DASHBOARD'])
    print(f"Тест M2.1 завершен. Сигнал: {test_m21.test_signal}. Отчет сохранен в m21_gini_test_report.html")
except Exception as e:
    print(f"Ошибка в тесте M2.1: {e}")
    print("Пропускаем этот тест...")

# M2.2 - Gini по факторам
print("\n=== M2.2: Gini по факторам ===")

test_m22 = M22_GiniFactorsTest()
try:
    # M22_GiniFactorsTest.run() требует df_train, df_test, target_column, features
    df_train_with_target = X_train_cls.copy()
    df_train_with_target['target'] = y_train_cls
    
    df_test_with_target = X_test_cls.copy()
    df_test_with_target['target'] = y_test_cls
    
    result_m22 = test_m22.run(
        df_train=df_train_with_target, 
        df_test=df_test_with_target,
        target_column='target', 
        features=['Pclass', 'Sex_encoded', 'Age', 'Fare']
    )
    with open('m22_gini_factors_report.html', 'w', encoding='utf-8') as f:
        f.write(result_m22['DASHBOARD'])
    print(f"Тест M2.2 завершен. Сигнал: {test_m22.test_signal}. Отчет сохранен в m22_gini_factors_report.html")
except Exception as e:
    print(f"Ошибка в тесте M2.2: {e}")
    print("Пропускаем этот тест...")

# M2.4 - Динамика Gini (пропускаем M2.3, так как его нет в файле)
print("\n=== M2.4: Динамика Gini ===")

test_m24 = M24_GiniDynamicsTest()
try:
    # M24_GiniDynamicsTest.run() требует df_train, df_test, date_column, score_column, target_column
    # Создаем данные с датами и скорами для демонстрации
    train_data_with_dates = X_train_cls.copy()
    train_data_with_dates['date_column'] = pd.date_range('2023-01-01', periods=len(train_data_with_dates), freq='D')
    train_data_with_dates['score'] = y_pred_proba_train_cls  # правильный размер для train
    train_data_with_dates['target'] = y_train_cls
    
    test_data_with_dates = X_test_cls.copy()
    test_data_with_dates['date_column'] = pd.date_range('2023-06-01', periods=len(test_data_with_dates), freq='D')
    test_data_with_dates['score'] = y_pred_proba_cls  # правильный размер для test
    test_data_with_dates['target'] = y_test_cls
    
    result_m24 = test_m24.run(
        df_train=train_data_with_dates, 
        df_test=test_data_with_dates,
        date_column='date_column', 
        score_column='score', 
        target_column='target'
    )
    with open('m24_gini_dynamics_report.html', 'w', encoding='utf-8') as f:
        f.write(result_m24['DASHBOARD'])
    print(f"Тест M2.4 завершен. Сигнал: {test_m24.test_signal}. Отчет сохранен в m24_gini_dynamics_report.html")
except Exception as e:
    print(f"Ошибка в тесте M2.4: {e}")
    print("Пропускаем этот тест...")

# M2.5 - Gini Uplift тест
print("\n=== M2.5: Gini Uplift тест ===")

test_m25 = M25_GiniUpliftTest()
try:
    # M25_GiniUpliftTest.run() требует df_train, df_test, target_column, feature_order
    result_m25 = test_m25.run(
        df_train=df_train_with_target,
        df_test=df_test_with_target,
        target_column='target',
        feature_order=['Pclass', 'Sex_encoded', 'Age', 'Fare'],
        n_jobs=3,
        m22_test_object=test_m22  # передаем предыдущий тест для переиспользования Gini
    )
    with open('m25_gini_uplift_report.html', 'w', encoding='utf-8') as f:
        f.write(result_m25['combined'])
    print(f"Тест M2.5 завершен. Сигнал: {test_m25.test_signal}. Отчет сохранен в m25_gini_uplift_report.html")
except Exception as e:
    print(f"Ошибка в тесте M2.5: {e}")
    print("Пропускаем этот тест...")

=== M2.1: Gini тест ===
Тест M2.1 завершен. Сигнал: green. Отчет сохранен в m21_gini_test_report.html

=== M2.2: Gini по факторам ===
Тест M2.2 завершен. Сигнал: red. Отчет сохранен в m22_gini_factors_report.html

=== M2.4: Динамика Gini ===
Тест M2.2 завершен. Сигнал: red. Отчет сохранен в m22_gini_factors_report.html

=== M2.4: Динамика Gini ===
Тест M2.4 завершен. Сигнал: green. Отчет сохранен в m24_gini_dynamics_report.html

=== M2.5: Gini Uplift тест ===
Тест M2.5 завершен. Сигнал: red. Отчет сохранен в m25_gini_uplift_report.html
Тест M2.4 завершен. Сигнал: green. Отчет сохранен в m24_gini_dynamics_report.html

=== M2.5: Gini Uplift тест ===
Тест M2.5 завершен. Сигнал: red. Отчет сохранен в m25_gini_uplift_report.html


### 6.4 Модуль M3 - Мультиколлинеарность

In [16]:
# M3.3 - Тест мультиколлинеарности
print("=== M3.3: Тест мультиколлинеарности ===")

test_m33 = M33_MulticollinearityTest()
try:
    result_m33 = test_m33.run(X=X_train_cls)
    with open('m33_multicollinearity_report.html', 'w', encoding='utf-8') as f:
        f.write(result_m33['report'])
    print(f"Тест M3.3 завершен. Сигнал: {test_m33.test_signal}. Отчет сохранен в m33_multicollinearity_report.html")
except Exception as e:
    print(f"Ошибка в тесте M3.3: {e}")
    print("Пропускаем этот тест...")

print("\nПримечание: Другие тесты M3 (M3.1, M3.2, M3.4, M3.5) не реализованы в текущей версии файлов.")

=== M3.3: Тест мультиколлинеарности ===
Ошибка в тесте M3.3: M33_MulticollinearityTest.run() got an unexpected keyword argument 'X'
Пропускаем этот тест...

Примечание: Другие тесты M3 (M3.1, M3.2, M3.4, M3.5) не реализованы в текущей версии файлов.


### 6.5 Модуль M5 - Стабильность модели

In [17]:
# M5.1 - Стабильность Gini модели
print("=== M5.1: Стабильность Gini модели ===")

test_m51 = M51_ModelGiniStabilityTest()
try:
    result_m51 = test_m51.run(
        score_train=pd.Series(y_pred_proba_cls[:len(y_train_cls)]), 
        target_train=y_train_cls,
        score_test=pd.Series(y_pred_proba_cls[len(y_train_cls):]), 
        target_test=y_test_cls
    )
    with open('m51_model_gini_stability_report.html', 'w', encoding='utf-8') as f:
        f.write(result_m51['report'])
    print(f"Тест M5.1 завершен. Сигнал: {test_m51.test_signal}. Отчет сохранен в m51_model_gini_stability_report.html")
except Exception as e:
    print(f"Ошибка в тесте M5.1: {e}")
    print("Пропускаем этот тест...")

# M5.2 - Стабильность Gini факторов
print("\n=== M5.2: Стабильность Gini факторов ===")

test_m52 = M52_FactorGiniStabilityTest()
try:
    # Создаем DataFrame с целевой переменной для обучения и теста
    df_train_with_target = X_train_cls.copy()
    df_train_with_target['target'] = y_train_cls
    
    df_test_with_target = X_test_cls.copy() 
    df_test_with_target['target'] = y_test_cls
    
    result_m52 = test_m52.run(
        df_train=df_train_with_target, 
        df_test=df_test_with_target,
        target_column='target', 
        features=['Pclass', 'Sex_encoded', 'Age', 'Fare']
    )
    with open('m52_factor_gini_stability_report.html', 'w', encoding='utf-8') as f:
        f.write(result_m52['report'])
    print(f"Тест M5.2 завершен. Сигнал: {test_m52.test_signal}. Отчет сохранен в m52_factor_gini_stability_report.html")
except Exception as e:
    print(f"Ошибка в тесте M5.2: {e}")
    print("Пропускаем этот тест...")

# M5.3 - PSI тест скоров
print("\n=== M5.3: PSI тест скоров ===")

test_m53 = M53_ScorePSITest(psi_calc=psi_calc)
try:
    # Создаем DataFrame для обучения и теста с скорами и целевой переменной
    train_df = pd.DataFrame({
        'score': y_pred_proba_cls[:len(y_train_cls)],
        'target': y_train_cls
    })
    
    oot_df = pd.DataFrame({
        'score': y_pred_proba_cls[len(y_train_cls):],
        'target': y_test_cls
    })
    
    result_m53 = test_m53.run(
        train_df=train_df, 
        oot_df=oot_df, 
        score_column='score', 
        target_column='target'
    )
    with open('m53_score_psi_report.html', 'w', encoding='utf-8') as f:
        f.write(result_m53['report'])
    print(f"Тест M5.3 завершен. Сигнал: {test_m53.test_signal}. Отчет сохранен в m53_score_psi_report.html")
except Exception as e:
    print(f"Ошибка в тесте M5.3: {e}")
    print("Пропускаем этот тест...")

# M5.4 - PSI тест факторов
print("\n=== M5.4: PSI тест факторов ===")

test_m54 = M54_FactorPSITest(psi_calc=psi_calc)
try:
    result_m54 = test_m54.run(
        train_df=df_train_with_target,
        oot_df=df_test_with_target, 
        target_column='target',
        features=['Pclass', 'Sex_encoded', 'Age', 'Fare'],
        categorical_features=['Pclass', 'Sex_encoded']
    )
    with open('m54_factor_psi_report.html', 'w', encoding='utf-8') as f:
        f.write(result_m54['report'])
    print(f"Тест M5.4 завершен. Сигнал: {test_m54.test_signal}. Отчет сохранен в m54_factor_psi_report.html")
except Exception as e:
    print(f"Ошибка в тесте M5.4: {e}")
    print("Пропускаем этот тест...")

=== M5.1: Стабильность Gini модели ===
Ошибка в тесте M5.1: Found input variables with inconsistent numbers of samples: [712, 179]
Пропускаем этот тест...

=== M5.2: Стабильность Gini факторов ===


M 5.2 - Processing Features: 100%|██████████| 4/4 [00:00<00:00, 363.65it/s]


Ошибка в тесте M5.2: 'report'
Пропускаем этот тест...

=== M5.3: PSI тест скоров ===
Ошибка в тесте M5.3: array length 179 does not match index length 712
Пропускаем этот тест...

=== M5.4: PSI тест факторов ===


M 5.4 - Processing Features: 100%|██████████| 4/4 [00:02<00:00,  1.90it/s]

Ошибка в тесте M5.4: 'report'
Пропускаем этот тест...


### 6.6 Модуль M4 - Качество работы алгоритма

#### 6.6.1 Раздел 4.1 - Качество работы алгоритма

In [18]:
# M4.1 - Confidence Interval of Quality Metric (Classification)
print("=== M4.1: Confidence Interval of Quality Metric (Classification) ===")

test_m41_cls = M41_MetricConfidenceIntervalTest(model_type='classification', n_iterations=100)
result_m41_cls = test_m41_cls.run(y_true=y_test_cls, y_pred=y_pred_cls)

# Save result for report
with open('m41_classification_report.html', 'w', encoding='utf-8') as f:
    f.write(result_m41_cls['report'])
    
print("Test M4.1 (classification) completed. Result saved to m41_classification_report.html")

=== M4.1: Confidence Interval of Quality Metric (Classification) ===


Bootstrapping metric: 100%|██████████| 100/100 [00:00<00:00, 624.62it/s]


Test M4.1 (classification) completed. Result saved to m41_classification_report.html


In [19]:
# M4.1 - Confidence Interval of Quality Metric (Regression)
print("=== M4.1: Confidence Interval of Quality Metric (Regression) ===")

test_m41_reg = M41_MetricConfidenceIntervalTest(model_type='regression', n_iterations=100)
result_m41_reg = test_m41_reg.run(y_true=y_test_reg, y_pred=y_pred_reg)

with open('m41_regression_report.html', 'w', encoding='utf-8') as f:
    f.write(result_m41_reg['report'])
    
print("Test M4.1 (regression) completed. Result saved to m41_regression_report.html")

=== M4.1: Confidence Interval of Quality Metric (Regression) ===


Bootstrapping metric: 100%|██████████| 100/100 [00:00<00:00, 1666.73it/s]

Test M4.1 (regression) completed. Result saved to m41_regression_report.html


In [20]:
# M4.2 - Check for Insufficient Number of Observations
print("=== M4.2: Check for Insufficient Number of Observations ===")

test_m42 = M42_SampleSizeAdequacyTest(model_type='classification', step=0.2)
result_m42 = test_m42.run(X=X_train_cls, y=y_train_cls, model=CatBoostClassifier(verbose=False, iterations=50))

with open('m42_sample_size_report.html', 'w', encoding='utf-8') as f:
    f.write(result_m42['report'])
    
print("Test M4.2 completed. Result saved to m42_sample_size_report.html")

=== M4.2: Check for Insufficient Number of Observations ===


Testing different sample sizes: 100%|██████████| 5/5 [00:00<00:00, 14.90it/s]

Test M4.2 completed. Result saved to m42_sample_size_report.html


In [21]:
# M4.3 - Model Calibration (classification only)
print("=== M4.3: Model Calibration ===")

test_m43 = M43_CalibrationTest(n_bins=10)
result_m43 = test_m43.run(y_true=y_test_cls, y_prob=y_pred_proba_cls)

with open('m43_calibration_report.html', 'w', encoding='utf-8') as f:
    f.write(result_m43['report'])
    
print("Test M4.3 completed. Result saved to m43_calibration_report.html")

=== M4.3: Model Calibration ===
Test M4.3 completed. Result saved to m43_calibration_report.html


#### 6.6.2 Раздел 4.2 - Стабильность результатов

In [22]:
# M4.4 - Overfitting Detection
print("=== M4.4: Overfitting Detection ===")

# Create dummy scores for train and test
from sklearn.metrics import f1_score
train_scores = [f1_score(y_train_cls, catboost_cls.predict(X_train_cls))] * 5
test_scores = [f1_score(y_test_cls, y_pred_cls)] * 5

test_m44 = M44_OverfittingTest(model_type='classification')
result_m44 = test_m44.run(train_scores=train_scores, test_scores=test_scores)

with open('m44_overfitting_report.html', 'w', encoding='utf-8') as f:
    f.write(result_m44['report'])
    
print(f"Test M4.4 completed. Signal: {test_m44.test_signal}. Result saved to m44_overfitting_report.html")

=== M4.4: Overfitting Detection ===
Test M4.4 completed. Signal: orange. Result saved to m44_overfitting_report.html


In [23]:
# M4.5 - Metric Stability Over Time
print("=== M4.5: Metric Stability Over Time ===")

# Create dummy time series
dates = pd.date_range('2023-01-01', periods=12, freq='M')
scores = np.random.normal(0.85, 0.05, 12)  # Simulate metrics with small volatility

test_m45 = M45_MetricStabilityTest()
result_m45 = test_m45.run(dates=dates, scores=pd.Series(scores))

with open('m45_stability_report.html', 'w', encoding='utf-8') as f:
    f.write(result_m45['report'])
    
print("Test M4.5 completed. Result saved to m45_stability_report.html")

=== M4.5: Metric Stability Over Time ===
Test M4.5 completed. Result saved to m45_stability_report.html
Test M4.5 completed. Result saved to m45_stability_report.html


In [24]:
# M4.6 - Quality by Categories
print("=== M4.6: Quality by Categories ===")

# Create dataframe with predictions and categories
category_df = X_test_cls.copy()
category_df['target'] = y_test_cls
category_df['predictions'] = y_pred_cls
category_df['Pclass_category'] = category_df['Pclass']  # Use passenger class as category

test_m46 = M46_CategoryQualityTest(model_type='classification')
result_m46 = test_m46.run(
    df=category_df, 
    category_column='Pclass_category', 
    target_column='target', 
    pred_column='predictions'
)

with open('m46_category_report.html', 'w', encoding='utf-8') as f:
    f.write(result_m46['report'])
    
print("Test M4.6 completed. Result saved to m46_category_report.html")

=== M4.6: Quality by Categories ===
Test M4.6 completed. Result saved to m46_category_report.html
Test M4.6 completed. Result saved to m46_category_report.html


#### 6.6.3 Раздел 4.3 - Качество отбора признаков

In [25]:
# M4.7 - SHAP Importance Comparison (may be slow)
print("=== M4.7: SHAP Importance Comparison ===")

test_m47 = M47_ShapImportanceTest()
result_m47 = test_m47.run(
    model=catboost_cls, 
    X_train=X_train_cls.sample(200),  # Reduce size for demonstration
    X_test=X_test_cls.sample(100), 
    top_n=10
)

with open('m47_shap_report.html', 'w', encoding='utf-8') as f:
    f.write(result_m47['report'])
    
print("Test M4.7 completed. Result saved to m47_shap_report.html")

=== M4.7: SHAP Importance Comparison ===
Test M4.7 completed. Result saved to m47_shap_report.html
Test M4.7 completed. Result saved to m47_shap_report.html


In [26]:
# M4.8 - Feature Contribution Adequacy
print("=== M4.8: Feature Contribution Adequacy ===")

feature_descriptions = {
    'Pclass': 'Passenger class (1, 2, 3)',
    'Sex_encoded': 'Passenger gender (0 - female, 1 - male)',
    'Age': 'Passenger age',
    'Fare': 'Ticket price',
    'Embarked_encoded': 'Port of embarkation',
    'FamilySize': 'Family size',
    'IsAlone': 'Traveling alone (1 - yes, 0 - no)',
    'Title': 'Passenger title'
}

test_m48 = M48_FeatureContributionTest()
result_m48 = test_m48.run(
    model=catboost_cls, 
    X_sample=X_test_cls.sample(100),
    feature_descriptions=feature_descriptions
)

with open('m48_contribution_report.html', 'w', encoding='utf-8') as f:
    f.write(result_m48['report'])
    
print("Test M4.8 completed. Result saved to m48_contribution_report.html")

=== M4.8: Feature Contribution Adequacy ===
Test M4.8 completed. Result saved to m48_contribution_report.html
Test M4.8 completed. Result saved to m48_contribution_report.html


In [27]:
# M4.9 - Uplift Test (may be slow)
print("=== M4.9: Uplift Test ===")

# Use smaller sample for demonstration
sample_size = 300
X_train_small = X_train_cls.sample(sample_size)
y_train_small = y_train_cls.loc[X_train_small.index]
X_test_small = X_test_cls.sample(100)
y_test_small = y_test_cls.loc[X_test_small.index]

test_m49 = M49_UpliftTest(model_type='classification', step=2)
result_m49 = test_m49.run(
    X_train=X_train_small,
    y_train=y_train_small,
    X_test=X_test_small,
    y_test=y_test_small,
    model_class=lambda: CatBoostClassifier(verbose=False, iterations=50)
)

with open('m49_uplift_report.html', 'w', encoding='utf-8') as f:
    f.write(result_m49['report'])
    
print(f"Test M4.9 completed. Signal: {test_m49.test_signal}. Result saved to m49_uplift_report.html")

=== M4.9: Uplift Test ===


Uplift test: 100%|██████████| 4/4 [00:00<00:00, 15.06it/s]



Test M4.9 completed. Signal: orange. Result saved to m49_uplift_report.html


In [28]:
# M4.10 - Признаки с большим количеством пропущенных значений
print("=== M4.10: Признаки с большим количеством пропущенных значений ===")

# Создаем демонстрационный датасет с пропущенными значениями
X_demo = X_train_cls.sample(300).copy()

# Искусственно добавляем много пропусков в некоторые признаки для демонстрации
# Делаем Age с очень высоким уровнем пропусков (>80%)
missing_indices_age = X_demo.sample(int(0.85 * len(X_demo))).index
X_demo.loc[missing_indices_age, 'Age'] = np.nan

# Делаем Fare с умеренным уровнем пропусков (~60%)
missing_indices_fare = X_demo.sample(int(0.6 * len(X_demo))).index
X_demo.loc[missing_indices_fare, 'Fare'] = np.nan

# Добавляем новый признак с очень высоким уровнем пропусков
X_demo['problematic_feature'] = np.nan
valid_indices = X_demo.sample(int(0.1 * len(X_demo))).index  # только 10% значений
X_demo.loc[valid_indices, 'problematic_feature'] = np.random.normal(0, 1, len(valid_indices))

# Добавляем столбец времени для тепловой карты
n_rows = len(X_demo)
date_range = pd.date_range('2024-01-01', periods=5, freq='M')  # 3 месяца
X_demo['date_demo'] = np.random.choice(date_range, size=n_rows, replace=True)

print(f"Создан демонстрационный датасет размером: {X_demo.shape}")
print("Доли пропущенных значений по признакам:")
missing_stats = X_demo.isnull().mean().sort_values(ascending=False)
for feature, ratio in missing_stats.head(5).items():
    if feature != 'date_demo':
        print(f"  {feature}: {ratio:.1%}")

# Запуск теста M4.10
test_m410 = M410_MissingValuesImpactTest(threshold=0.1)
result_m410 = test_m410.run(
    X=X_demo,
    time_column='date_demo'
)

# Сохраняем отчет
with open('m410_missing_values_report.html', 'w', encoding='utf-8') as f:
    f.write(result_m410['report'])

print(f"\n✅ Тест M4.10 завершен!")
print(f"🚦 Сигнал: {test_m410.test_signal.upper()}")
print(f"📄 Отчет сохранен в: m410_missing_values_report.html")
print("\n📊 Тест включает:")
print("   • Столбчатую диаграмму признаков с высоким уровнем пропусков")
print("   • Детальную таблицу с процентами пропущенных значений") 
print("   • Тепловую карту пропусков по времени (время по оси X, признаки по оси Y)")
print("   • Светофорную оценку качества данных")

=== M4.10: Признаки с большим количеством пропущенных значений ===
Создан демонстрационный датасет размером: (300, 10)
Доли пропущенных значений по признакам:
  problematic_feature: 90.0%
  Age: 85.0%
  Fare: 60.0%
  Pclass: 0.0%
  Sex_encoded: 0.0%

✅ Тест M4.10 завершен!
🚦 Сигнал: GREEN
📄 Отчет сохранен в: m410_missing_values_report.html

📊 Тест включает:
   • Столбчатую диаграмму признаков с высоким уровнем пропусков
   • Детальную таблицу с процентами пропущенных значений
   • Тепловую карту пропусков по времени (время по оси X, признаки по оси Y)
   • Светофорную оценку качества данных

✅ Тест M4.10 завершен!
🚦 Сигнал: GREEN
📄 Отчет сохранен в: m410_missing_values_report.html

📊 Тест включает:
   • Столбчатую диаграмму признаков с высоким уровнем пропусков
   • Детальную таблицу с процентами пропущенных значений
   • Тепловую карту пропусков по времени (время по оси X, признаки по оси Y)
   • Светофорную оценку качества данных


#### 6.6.4 Раздел 4.4 - Альтернативный отбор факторов

In [29]:
# M4.11 - Feature Correlation with Target
print("=== M4.11: Feature Correlation with Target ===")

test_m411 = M411_TargetCorrelationTest()
result_m411 = test_m411.run(X=X_train_cls, y=y_train_cls)

with open('m411_target_corr_report.html', 'w', encoding='utf-8') as f:
    f.write(result_m411['report'])

print("Test M4.11 completed. Result saved to m411_target_corr_report.html")

=== M4.11: Feature Correlation with Target ===
Test M4.11 completed. Result saved to m411_target_corr_report.html


In [30]:
# M4.12 - Feature Correlation Among Themselves
print("=== M4.12: Feature Correlation Among Themselves ===")

test_m412 = M412_FeatureCorrelationTest(threshold=0.8)
result_m412 = test_m412.run(X=X_train_cls)

with open('m412_feature_corr_report.html', 'w', encoding='utf-8') as f:
    f.write(result_m412['report'])
    
print("Test M4.12 completed. Result saved to m412_feature_corr_report.html")

=== M4.12: Feature Correlation Among Themselves ===
Test M4.12 completed. Result saved to m412_feature_corr_report.html
Test M4.12 completed. Result saved to m412_feature_corr_report.html


In [31]:
# M4.13 - VIF анализ
print("=== M4.13: VIF анализ ===")

test_m413 = M413_VIFTest()
result_m413 = test_m413.run(X=X_train_cls)

with open('m413_vif_report.html', 'w', encoding='utf-8') as f:
    f.write(result_m413['report'])
    
print("Тест M4.13 выполнен. Результат сохранен в m413_vif_report.html")

=== M4.13: VIF анализ ===
Тест M4.13 выполнен. Результат сохранен в m413_vif_report.html


In [32]:
# M4.14 - Двухлесовой отбор (может быть медленным)
print("=== M4.14: Двухлесовой отбор признаков ===")

# Используем меньшую выборку и меньше итераций для демонстрации
X_sample = X_train_cls.sample(300)
y_sample = y_train_cls.loc[X_sample.index]

test_m414 = M414_TwoForestSelectionTest(model_type='classification', n_iterations=20)
result_m414 = test_m414.run(X=X_sample, y=y_sample)

with open('m414_two_forest_report.html', 'w', encoding='utf-8') as f:
    f.write(result_m414['report'])
    
print("Тест M4.14 выполнен. Результат сохранен в m414_two_forest_report.html")

=== M4.14: Двухлесовой отбор признаков ===


Two-forest selection: 100%|██████████| 8/8 [00:06<00:00,  1.16it/s]

Тест M4.14 выполнен. Результат сохранен в m414_two_forest_report.html


## 7. Создание полного отчета по всем модулям M0-M5

In [ ]:
# Создание полного итогового отчета с использованием TestGroupBuilder
print("=== СОЗДАНИЕ ИТОГОВОГО ОТЧЕТА ЧЕРЕЗ TESTGROUPBUILDER ===")

try:
    from core.base_test import TestGroupBuilder, ModelReportBuilder
    
    # Создаем группы тестов для каждого модуля
    groups = {}
    
    # M0 - Качество данных
    print("Создаем группу M0...")
    m0_group = TestGroupBuilder('M0', 'Качество данных')
    try:
        m0_group.add_test(test_m01)
        m0_group.add_test(test_m02) 
        m0_group.add_test(test_m03)
    except Exception as e:
        print(f"Ошибка добавления тестов M0: {e}")
    groups['M0'] = m0_group
    
    # M1 - Тесты распределения
    print("Создаем группу M1...")
    m1_group = TestGroupBuilder('M1', 'Тесты распределения')
    try:
        m1_group.add_test(test_m11)
        m1_group.add_test(test_m12)
        m1_group.add_test(test_m13)
        m1_group.add_test(test_m14)
        m1_group.add_test(test_m15)
    except Exception as e:
        print(f"Ошибка добавления тестов M1: {e}")
    groups['M1'] = m1_group
    
    # M2 - Gini тесты
    print("Создаем группу M2...")
    m2_group = TestGroupBuilder('M2', 'Gini тесты')
    try:
        m2_group.add_test(test_m21)
        m2_group.add_test(test_m22)
        m2_group.add_test(test_m24)
        m2_group.add_test(test_m25)
    except Exception as e:
        print(f"Ошибка добавления тестов M2: {e}")
    groups['M2'] = m2_group
    
    # M3 - Мультиколлинеарность
    print("Создаем группу M3...")
    m3_group = TestGroupBuilder('M3', 'Мультиколлинеарность')
    try:
        m3_group.add_test(test_m33)
    except Exception as e:
        print(f"Ошибка добавления тестов M3: {e}")
    groups['M3'] = m3_group
    
    # M4 - Качество работы алгоритма
    print("Создаем группу M4...")
    m4_group = TestGroupBuilder('M4', 'Качество работы алгоритма')
    try:
        m4_group.add_test(test_m41_cls)
        m4_group.add_test(test_m41_reg)
        m4_group.add_test(test_m42)
        m4_group.add_test(test_m43)
        m4_group.add_test(test_m44)
        m4_group.add_test(test_m45)
        m4_group.add_test(test_m46)
        m4_group.add_test(test_m47)
        m4_group.add_test(test_m48)
        m4_group.add_test(test_m49)
        m4_group.add_test(test_m410)
        m4_group.add_test(test_m411)
        m4_group.add_test(test_m412)
        m4_group.add_test(test_m413)
        m4_group.add_test(test_m414)
    except Exception as e:
        print(f"Ошибка добавления тестов M4: {e}")
    groups['M4'] = m4_group
    
    # M5 - Стабильность модели
    print("Создаем группу M5...")
    m5_group = TestGroupBuilder('M5', 'Стабильность модели')
    try:
        m5_group.add_test(test_m51)
        m5_group.add_test(test_m52)
        m5_group.add_test(test_m53)
        m5_group.add_test(test_m54)
    except Exception as e:
        print(f"Ошибка добавления тестов M5: {e}")
    groups['M5'] = m5_group
    
    # Создаем основной отчет
    print("\nСоздаем основной отчет...")
    main_report = ModelReportBuilder('SBS Evaluation Tool - Полный отчет M0-M5')
    
    # Добавляем все группы в отчет
    for group_id, group in groups.items():
        try:
            main_report.add_group(group)
            print(f"✅ Группа {group_id} добавлена в отчет")
        except Exception as e:
            print(f"❌ Ошибка добавления группы {group_id}: {e}")
    
    # Генерируем итоговый HTML отчет
    print("\nГенерируем итоговый HTML отчет...")
    final_html = main_report.generate_report()
    
    # Сохраняем отчет
    with open('sbs_complete_report_m0_m5.html', 'w', encoding='utf-8') as f:
        f.write(final_html)
    
    print("✅ Итоговый отчет создан и сохранен в: sbs_complete_report_m0_m5.html")
    
    # Выводим сводную статистику
    print("\n" + "="*80)
    print("📊 СВОДНАЯ СТАТИСТИКА ПО МОДУЛЯМ:")
    print("="*80)
    
    total_tests = 0
    total_green = 0
    total_yellow = 0
    total_red = 0
    
    for group_id, group in groups.items():
        group_stats = group.get_signal_summary()
        total_tests += group_stats['total']
        total_green += group_stats.get('green', 0)
        total_yellow += group_stats.get('yellow', 0) + group_stats.get('orange', 0)  # orange считаем как yellow
        total_red += group_stats.get('red', 0)
        
        print(f"🔸 {group.group_name}: {group_stats['total']} тестов")
        print(f"   🟢 Зеленых: {group_stats.get('green', 0)}")
        print(f"   🟡 Желтых/Оранжевых: {group_stats.get('yellow', 0) + group_stats.get('orange', 0)}")
        print(f"   🔴 Красных: {group_stats.get('red', 0)}")
        print()
    
    print("="*80)
    print(f"🎯 ИТОГО:")
    print(f"📊 Всего тестов: {total_tests}")
    print(f"✅ Зеленых: {total_green}")
    print(f"⚠️ Желтых/Оранжевых: {total_yellow}")
    print(f"❌ Красных: {total_red}")
    print("="*80)
    
    # Процентное соотношение
    if total_tests > 0:
        print(f"? Процентное соотношение:")
        print(f"✅ Зеленые: {total_green/total_tests*100:.1f}%")
        print(f"⚠️ Желтые/Оранжевые: {total_yellow/total_tests*100:.1f}%")
        print(f"❌ Красные: {total_red/total_tests*100:.1f}%")
    
    print("\n🎉 ПОЛНЫЙ ОТЧЕТ SBS EVALUATION TOOL СОЗДАН!")
    
except Exception as e:
    print(f"❌ Ошибка создания итогового отчета: {e}")
    print("Используем упрощенную версию без TestGroupBuilder...")
    
    # Упрощенная версия без TestGroupBuilder
    print("\n=== УПРОЩЕННАЯ СВОДНАЯ СТАТИСТИКА ===")
    
    all_tests = {
        'M0': [test_m01, test_m02, test_m03],
        'M1': [test_m11, test_m12, test_m13, test_m14, test_m15],
        'M2': [test_m21, test_m22, test_m24, test_m25],
        'M3': [test_m33],
        'M4': [test_m41_cls, test_m41_reg, test_m42, test_m43, test_m44, test_m45, 
               test_m46, test_m47, test_m48, test_m49, test_m410, test_m411, 
               test_m412, test_m413, test_m414],
        'M5': [test_m51, test_m52, test_m53, test_m54]
    }
    
    for module, tests in all_tests.items():
        print(f"\n🔸 Модуль {module}:")
        for i, test in enumerate(tests):
            try:
                signal = getattr(test, 'test_signal', 'НЕТ СИГНАЛА')
                print(f"   Тест {i+1}: {signal}")
            except:
                print(f"   Тест {i+1}: НЕТ СИГНАЛА")

## 8. Заключение и рекомендации по всем модулям

In [34]:
print("=== ЗАКЛЮЧЕНИЕ ПО ВСЕМ МОДУЛЯМ (ОБНОВЛЕННОЕ) ===")
print("\n✅ Полная демонстрация завершена с исправлениями!")

print("\n📊 Протестированные модули:")
print("• M0: Качество данных (3 теста)")
print("• M1: Тесты распределения (5 тестов)")  
print("• M2: Gini тесты (4 теста) ✅ ИСПРАВЛЕНО")
print("• M3: Мультиколлинеарность (1 тест)")
print("• M4: Качество работы алгоритма (15 тестов)")
print("• M5: Стабильность модели (4 теста)")
print(f"📈 Всего тестов: {3+5+4+1+15+4} = 32 теста")

print("\n🔧 Исправленные проблемы:")
print("• M2.1: Исправлены размеры данных для score_train и score_test")
print("• M2.4: Исправлено несоответствие размеров датафреймов") 
print("• Добавлены предсказания для train выборки: y_pred_proba_train_cls")

print("\n📋 Статус тестов M2:")
try:
    print(f"• M2.1 (Gini тест): {test_m21.test_signal}")
    print(f"• M2.2 (Gini по факторам): {test_m22.test_signal}")
    print(f"• M2.4 (Динамика Gini): {test_m24.test_signal}")
    print(f"• M2.5 (Gini Uplift): {test_m25.test_signal}")
except:
    print("• Некоторые тесты M2 все еще выполняются...")

print("\n📁 Созданные отчеты M2:")
import os
m2_files = [f for f in os.listdir('.') if f.startswith('m2') and f.endswith('.html')]
for i, file in enumerate(sorted(m2_files), 1):
    print(f"{i}. {file}")

print("\n🎯 Основные результаты:")
print(f"• Модель классификации (Titanic): F1-score = {f1_score(y_test_cls, y_pred_cls):.3f}")
print(f"• AUC для train: {roc_auc_score(y_train_cls, y_pred_proba_train_cls):.3f}")
print(f"• AUC для test: {roc_auc_score(y_test_cls, y_pred_proba_cls):.3f}")

print("\n⚡ Причины ошибок и их решения:")
print("🔍 Проблема: 'Found input variables with inconsistent numbers of samples: [712, 179]'")
print("✅ Решение: Получили предсказания для train выборки отдельно")
print("🔍 Проблема: 'Length of values (179) does not match length of index (712)'")  
print("✅ Решение: Исправили размеры данных в датафреймах для M2.4")

print("\n🚀 Все модули M0-M5 теперь работают корректно!")
print("📧 Тесты M2 успешно исправлены и выполнены")

=== ЗАКЛЮЧЕНИЕ ПО ВСЕМ МОДУЛЯМ (ОБНОВЛЕННОЕ) ===

✅ Полная демонстрация завершена с исправлениями!

📊 Протестированные модули:
• M0: Качество данных (3 теста)
• M1: Тесты распределения (5 тестов)
• M2: Gini тесты (4 теста) ✅ ИСПРАВЛЕНО
• M3: Мультиколлинеарность (1 тест)
• M4: Качество работы алгоритма (15 тестов)
• M5: Стабильность модели (4 теста)
📈 Всего тестов: 32 = 32 теста

🔧 Исправленные проблемы:
• M2.1: Исправлены размеры данных для score_train и score_test
• M2.4: Исправлено несоответствие размеров датафреймов
• Добавлены предсказания для train выборки: y_pred_proba_train_cls

📋 Статус тестов M2:
• M2.1 (Gini тест): green
• M2.2 (Gini по факторам): red
• M2.4 (Динамика Gini): green
• M2.5 (Gini Uplift): red

📁 Созданные отчеты M2:
1. m21_gini_test_report.html
2. m22_gini_factors_report.html
3. m24_gini_dynamics_report.html
4. m25_gini_uplift_report.html

🎯 Основные результаты:
• Модель классификации (Titanic): F1-score = 0.803
• AUC для train: 0.998
• AUC для test: 0.838

⚡ П

In [ ]:
# Исправление M2 тестов - получаем предсказания для train выборки
print("=== ИСПРАВЛЕНИЕ M2 ТЕСТОВ ===")

# Получаем предсказания для train выборки
y_pred_proba_train_cls = catboost_cls.predict_proba(X_train_cls)[:, 1]
print(f"Получены предсказания для train: {y_pred_proba_train_cls.shape}")
print(f"Имеются предсказания для test: {y_pred_proba_cls.shape}")

# M2.1 - Gini тест (исправленная версия)
print("\n=== M2.1: Gini тест (исправленная версия) ===")

test_m21 = M21_GiniTest()
try:
    result_m21 = test_m21.run(
        score_train=pd.Series(y_pred_proba_train_cls), 
        target_train=y_train_cls,
        score_test=pd.Series(y_pred_proba_cls),
        target_test=y_test_cls
    )
    with open('m21_gini_test_report.html', 'w', encoding='utf-8') as f:
        f.write(result_m21['DASHBOARD'])
    print(f"✅ Тест M2.1 завершен успешно. Сигнал: {test_m21.test_signal}. Отчет сохранен в m21_gini_test_report.html")
except Exception as e:
    print(f"❌ Ошибка в тесте M2.1: {e}")
    print("Пропускаем этот тест...")

# M2.4 - Динамика Gini (исправленная версия)
print("\n=== M2.4: Динамика Gini (исправленная версия) ===")

test_m24 = M24_GiniDynamicsTest()
try:
    # Создаем данные с правильными размерами
    train_data_fixed = X_train_cls.copy()
    train_data_fixed['date_column'] = pd.date_range('2023-01-01', periods=len(X_train_cls), freq='D')
    train_data_fixed['score'] = y_pred_proba_train_cls  # правильный размер для train
    train_data_fixed['target'] = y_train_cls
    
    test_data_fixed = X_test_cls.copy()
    test_data_fixed['date_column'] = pd.date_range('2023-06-01', periods=len(X_test_cls), freq='D')
    test_data_fixed['score'] = y_pred_proba_cls  # правильный размер для test
    test_data_fixed['target'] = y_test_cls
    
    result_m24 = test_m24.run(
        df_train=train_data_fixed, 
        df_test=test_data_fixed,
        date_column='date_column', 
        score_column='score', 
        target_column='target'
    )
    with open('m24_gini_dynamics_report.html', 'w', encoding='utf-8') as f:
        f.write(result_m24['DASHBOARD'])
    print(f"✅ Тест M2.4 завершен успешно. Сигнал: {test_m24.test_signal}. Отчет сохранен в m24_gini_dynamics_report.html")
except Exception as e:
    print(f"❌ Ошибка в тесте M2.4: {e}")
    print("Пропускаем этот тест...")

print("\n🎉 Исправление M2 тестов завершено!")

=== ИСПРАВЛЕНИЕ M2 ТЕСТОВ ===
Получены предсказания для train: (712,)
Имеются предсказания для test: (179,)

=== M2.1: Gini тест (исправленная версия) ===
✅ Тест M2.1 завершен успешно. Сигнал: green. Отчет сохранен в m21_gini_test_report.html

=== M2.4: Динамика Gini (исправленная версия) ===
✅ Тест M2.4 завершен успешно. Сигнал: green. Отчет сохранен в m24_gini_dynamics_report.html

🎉 Исправление M2 тестов завершено!
✅ Тест M2.4 завершен успешно. Сигнал: green. Отчет сохранен в m24_gini_dynamics_report.html

🎉 Исправление M2 тестов завершено!


In [ ]:
# 1. Инициализация тестов M0
from tests.m0_data_quality import M01_DataQualityTest, M02_DataQualityTest, M03_DataQualityTest
from core.base_test import TestGroupBuilder

m01 = M01_DataQualityTest()
m02 = M02_DataQualityTest()
m03 = M03_DataQualityTest()

# 2. Создание группы
group_m0 = TestGroupBuilder(group_code="M0", group_name="M 0. Data Quality Tests")
group_m0.add_test(m01)
group_m0.add_test(m02)
group_m0.add_test(m03)

# 3. Запуск всех тестов с передачей параметров
group_m0.run_all_tests(test_params={
    "M 0.1": {
        # Пример: передайте необходимые параметры для теста M0.1
        # "df": train_df,
        # "target_column": target,
    },
    "M 0.2": {
        # Пример: параметры для теста M0.2
        # "df": train_df,
        # "features": num_columns,
    },
    "M 0.3": {
        # Пример: параметры для теста M0.3
        # "df": train_df,
        # "target_column": target,
    }
})

# 4. Просмотр результатов
menu_html = group_m0.generate_group_menu()
content_html = group_m0.generate_group_content()
print('Группа тестов M0 выполнена. Меню и контент готовы для отчета.')

In [ ]:
# 1. Инициализация тестов M0
from tests.m0_data_quality import M01_DataSummaryTest, M02_SampleOverlayTest, M03_ObsVsPredTest
from core.base_test import TestGroupBuilder

# Пример данных (замените на ваши переменные)
df = train_df  # основной датафрейм
date_column = 'date'  # колонка с датой
sample_column = 'sample'  # колонка с сэмплами
primary_key = 'id'  # первичный ключ
predict_column = 'score'  # колонка с предсказаниями
target_column = 'target'  # колонка с фактом

# 2. Создание группы
group_m0 = TestGroupBuilder(group_code="M0", group_name="M 0. Data Quality Tests")
group_m0.add_test(M01_DataSummaryTest())
group_m0.add_test(M02_SampleOverlayTest())
group_m0.add_test(M03_ObsVsPredTest())

# 3. Запуск всех тестов с передачей параметров
group_m0.run_all_tests(test_params={
    "M 0.1": {
        "df": df,
        "date_column": date_column,
        "sample_column": sample_column,
    },
    "M 0.2": {
        "df": df,
        "primary_key": primary_key,
        "sample_column": sample_column,
    },
    "M 0.3": {
        "df": df,
        "predict_column": predict_column,
        "target_column": target_column,
        "sample_column": sample_column,
    }
})

# 4. Просмотр результатов
menu_html = group_m0.generate_group_menu()
content_html = group_m0.generate_group_content()
print('Группа тестов M0 выполнена. Меню и контент готовы для отчета.')

In [ ]:
# Приведение данных к единому формату для тестов M0
## Для бинарной классификации
df_cls = train_df_cls  # основной датафрейм для классификации
date_column_cls = 'date'  # колонка с датой
sample_column_cls = 'sample'  # колонка с пометкой train/test
primary_key_cls = 'id'  # первичный ключ
predict_column_cls = 'score'  # колонка с предсказаниями (например, вероятности или скоринг)
target_column_cls = 'target'  # колонка с фактом (0/1)

## Для регрессии
df_reg = train_df_reg  # основной датафрейм для регрессии
date_column_reg = 'date'  # колонка с датой
sample_column_reg = 'sample'  # колонка с пометкой train/test
primary_key_reg = 'id'  # первичный ключ
predict_column_reg = 'score'  # колонка с предсказаниями (например, прогноз значения)
target_column_reg = 'target'  # колонка с фактом (числовое значение)

In [ ]:
# Объединение train и test данных регрессии в единый DataFrame с метками и датой
import pandas as pd

# train
df_train_reg = X_train_reg.copy()
df_train_reg['target'] = y_train_reg
df_train_reg['score'] = None  # для train предсказания можно добавить, если есть
df_train_reg['sample'] = 'train'
df_train_reg['date'] = pd.Timestamp('2025-01-01')  # замените на вашу дату или колонку

# test
df_test_reg = X_test_reg.copy()
df_test_reg['target'] = y_test_reg
df_test_reg['score'] = y_pred_reg
df_test_reg['sample'] = 'test'
df_test_reg['date'] = pd.Timestamp('2025-01-02')  # замените на вашу дату или колонку

# объединяем
df_reg = pd.concat([df_train_reg, df_test_reg], ignore_index=True)

print(df_reg.head())